# Looking under the hood of PewPew Qube

<a href="https://github.com/qiskit-community/qiskit-camp-europe-19/issues/62">PewPew Qube</a> is a quantum adaption of the famous Rubik's cube to the PewPew handheld console. 
It has been developed at the <a href="https://qiskit.camp/europe/">Qiskit Camp Europe</a> hackathon in September 2019. 

In short, the goal is to transform a random initial state to a well-defined final state. 
For a Rubik's cube, this is done by rotating the six sides of a multi-colored cube until all sides of the cube uniformly colored.
For the PewPew Qube game, the player applies quantum gates to the state vector of a two-qubit system. 
These gates rotate the state vector around various directions on the two-qubit Bloch-(hyper)sphere.
The goal is to reach a specific target state.
A more detailed explanation of the game and hints how to solve it are given <a href="./documentation.ipynb">here</a>.

In this notebook, we discuss the source code of the PewPew Qube. 

In [ ]:
%matplotlib notebook

In [ ]:
# import pygame to run the PewPew emulator from a notebook
import pygame

# add the src directory to the search path to load modules
import sys
sys.path.insert(0, "../src/")

## Basic interaction with the hardware

In a first step, we discuss the basic working principle of the PewPew console at the example of the animated initialization screen of the PewPew Qube game.

The screen of the PewPew console consists of 8x8 LEDs that can display four different brightness values (0 = off, 1, 2, and 3).
Thus, a single frame is represented by a 8x8 tuple of integers, and animations are obtained by showing several frames one after another. 
To display an animated qiskit logo, we need the four different frames, which we store in the tuple *qiskit_images*

In [ ]:
qiskit_images = (
    (
        (0, 3, 3, 3, 3, 3, 3, 0),
        (3, 2, 0, 1, 1, 0, 0, 3),
        (3, 0, 2, 0, 0, 1, 0, 3),
        (3, 1, 0, 2, 0, 0, 1, 3),
        (3, 3, 0, 0, 2, 0, 3, 3),
        (3, 0, 3, 3, 3, 3, 0, 3),
        (3, 0, 0, 0, 0, 0, 2, 3),
        (0, 3, 3, 3, 3, 3, 3, 0)
    ),
    (
        (0, 3, 3, 3, 3, 3, 3, 0),
        (3, 0, 0, 2, 1, 0, 0, 3),
        (3, 0, 1, 2, 0, 1, 0, 3),
        (3, 1, 0, 0, 2, 0, 1, 3),
        (3, 3, 0, 0, 2, 0, 3, 3),
        (3, 0, 3, 3, 3, 3, 0, 3),
        (3, 0, 0, 0, 0, 2, 0, 3),
        (0, 3, 3, 3, 3, 3, 3, 0)
    ),
    (
        (0, 3, 3, 3, 3, 3, 3, 0),
        (3, 0, 0, 1, 2, 0, 0, 3),
        (3, 0, 1, 0, 2, 1, 0, 3),
        (3, 1, 0, 2, 0, 0, 1, 3),
        (3, 3, 0, 2, 0, 0, 3, 3),
        (3, 0, 3, 3, 3, 3, 0, 3),
        (3, 0, 2, 0, 0, 0, 0, 3),
        (0, 3, 3, 3, 3, 3, 3, 0)
    ),
    (
        (0, 3, 3, 3, 3, 3, 3, 0),
        (3, 0, 0, 1, 1, 0, 2, 3),
        (3, 0, 1, 0, 0, 2, 0, 3),
        (3, 1, 0, 0, 2, 0, 1, 3),
        (3, 3, 0, 2, 0, 0, 3, 3),
        (3, 0, 3, 3, 3, 3, 0, 3),
        (3, 2, 0, 0, 0, 0, 0, 3),
        (0, 3, 3, 3, 3, 3, 3, 0)
    )
)

The main loop of the program performs three tasks: 

 * First, it checks if any buttons are pressed. 
   This is done by a bitwise AND of the output of *pew.keys()* and predefined masks *pew.K_UP*, *pew.K_DOWN*, etc. 
   Each button is assigned to a certain level, whose number is stored in the variable *value*. 
 * Second, the main loop determines the next frame to be displayed in the animation and updates the screen. 
 * Finally, the program waits 100 milliseconds and repeats the loop unless a level has already been chosen. 
 
Here is the corresponding code:

In [ ]:
import pew

i = 0      # frame counter
value = 0  # selected level

pew.init() # initialize the PewPew console

# main loop
while not value:
    # check for pressed keys
    keys = pew.keys()
    if keys & pew.K_UP:
        value = 1
    elif keys & pew.K_RIGHT:
        value = 2
    elif keys & pew.K_DOWN:
        value = 3
    elif keys & pew.K_LEFT:
        value = 4
    
    # display the next frame
    animation = (0,0,1,1,2,2,3,3,2,2,1,1)
    i = (i + 1) % len(animation)
    screen = qiskit_images[animation[i]]
    pew.show(pew.Pix.from_iter(screen))
    
    # wait 0.1 seconds
    pew.tick(0.1)
    
# freeze the screen while a button is pressed
while pew.keys():
    pew.tick(0.1)

# the following is just necessary in a jupyter notebook:
pygame.display.quit()
pygame.quit()

At this point of the source code, the main loop has been terminated and the player has selected a specific level. Let's play!

But wait... Before loading the level, we must perform three intermediate steps: 
 * we initialize the random-number generator with the time that has passed since the PewPew console has been switched on. Otherwise, the random seed will always be the same and the levels start with identical states over and over again. 
 * to save RAM, we unload the *qiskit_images* tuple
 * we load a more sophisticated main loop that is used to executes the levels. 

Finally, we are now in a position to load the specified level. 
Each level is represented by a so-called *instruction set* (a class with a certain structure discussed below) that is passed as an argument to the new main loop:

In [ ]:
import random
import time

# initialize the random-number generator
random.seed(int(time.monotonic()*1000))

# unload unnecessary objects to save RAM
del qiskit_images

# load the new main loop 
from loop import main_loop

# execute the new main loop passing to it the seleced level
if value == 1:
    from rotations import instruction_set_XYZ
    main_loop(instruction_set_XYZ())
else:
    from instruction_sets import InstructionSet
    from displays import IBMQ
    main_loop(InstructionSet(level=value-2, goal=IBMQ))
    
# the following is just necessary in a jupyter notebook:
pygame.display.quit()
pygame.quit()

In the next section, we will have a closer look at the new main loop and its addional features. 

## Advanced interaction with the hardware

The new main loop has to fulfill two important additional requirements: 
 * If a button is pressed, the loop should not just quit as before. 
   Instead, it should update the screen and the level should continue. 
 * The buttons of the PewPew console must be debounced.

To fulfill the first task, we need a standardized way how to interact with the different levels. 
Therefore, all levels are instances of a certain class, which we call an *instruction set*. 
The instruction set of the chosen level is passed as the first argument of the main loop. 
In the next section, we will dive into the details of the instruction set.
For now, it is only important to know that 
 * the instruction set has a method *get_current_screen()* that return a frame to be displayed (*i.e.*, an 8x8 tuple of integers ranging from 0 to 3), and that
 * the instruction set has a method *key_pressed(value)* by which we can nofity the instruction set if a button has been pressed.

Here is the code of the new main loop: 

In [ ]:
def main_loop(ins):
    # initialize PewPew console
    pew.init()

    # Load start screens
    for start_screen in start_screens:
        pew.show(pew.Pix.from_iter(start_screen))
        pew.tick(0.2)
    pew.show(pew.Pix.from_iter(blank_screen))
    pew.tick(0.5)
    
    # display the first frame of the level
    pew.show(ins.get_current_screen())

    # flags used throughout the loop
    bool_loop = True
    old_keys = 0
    
    # new main loop
    while bool_loop:
        keys = pew.keys()
    
        if keys != 0 and keys != old_keys:
            # old_keys is necessary to debounce the buttons
            old_keys = keys
    
            # dispatch the pushed buttons
            if keys & pew.K_X:
                value = pew.K_X
            elif keys & pew.K_DOWN:
                value = pew.K_DOWN
            elif keys & pew.K_LEFT:
                value = pew.K_LEFT
            elif keys & pew.K_RIGHT:
                value = pew.K_RIGHT
            elif keys & pew.K_UP:
                value = pew.K_UP
            elif keys & pew.K_O:
                # the key "O" ("Z" in the emulator) will terminate the game
                value = pew.K_O
                bool_loop = False
            else:
                value = 0

            # send the pressed key to the instruction set
            ins.key_pressed(value)
    
        elif keys == 0:
            # this is necessary to be able to push 
            # a button twice in a row
            old_keys = keys
    
        # update the screen and wait for 20ms
        pew.show(ins.get_current_screen())
        pew.tick(0.02)
        
    # the program has been terminated. 
    # display the final sequence
    for final_screen in final_screens:
        pew.show(pew.Pix.from_iter(final_screen))
        pew.tick(0.2)
    pew.show(pew.Pix.from_iter(blank_screen))
    pew.tick(0.2)

The new main loop starts very similar to the basic one introduced in the first section: 
It displays an animation whose frames are defined in the tuple *start_screen*.
It can be found in the file <a href="../src/loop.py">loop.py</a>. 

Next, it takes retrieves the first frame of the current level and displays it. 

Analogous to the simple loop discussed in the previous section, the while loop monitors the buttons of the PewPew console and updates the screen. 

However, the buttons now must be debounced, *i.e.*, only a single key-pressed event should be raised if the button is pressed, and holding the button down for longer should not raise more key-pressed events. To achieve this, we introduce an additional variable *old_keys* that stores the last pushed button. 
A key-pressed event is only processed if the pushed button differs from the previous one stored in *old_keys*.

If a new button has been pressed, this information is passed to the instruction set via the *key_pressed* method, a new frame is obtained using the *get_current_screen* method, and the display is updated.

Upon termination of the main loop, a farewell animation is displayed, which is stored in the tuple *final_screen*. 

## Defining levels

Finally, we are prepared to discuss the most important point of this tutorial: How to define a level of the PewPew Qube game. 
Here is the instruction set of level 4, which is defined in the file <a href="../scr/rotations.py">rotations.py</a>: 

In [ ]:
class instruction_set_XYZ:
    
    def __init__(self):
        # history of pushed keys
        self.key_hist = []
        # current state vector
        self.state = random_state()

    def key_pressed(self, key):
        if key == pew.K_UP:
            # forget all pushed buttons
            self.key_hist = []
        elif key == pew.K_LEFT or key == pew.K_DOWN or key == pew.K_RIGHT:
            # append button to history
            self.key_hist.append(key)
            
            # if two buttons have been pressed, determine the corresponding transformation
            if len(self.key_hist) == 2:
                if self.key_hist[0] == pew.K_LEFT:
                    gate = 'x'
                elif self.key_hist[0] == pew.K_DOWN:
                    gate = 'y'
                else:
                    gate = 'z'
                
                if self.key_hist[1] == pew.K_LEFT:
                    gate = gate + 'x'
                elif self.key_hist[1] == pew.K_DOWN:
                    gate = gate + 'y'
                else:
                    gate = gate + 'z'
                
                # update the state vector
                self.state = propagate_statevector(self.state, make_circuit(gate))
                
                # clear the history of pushed buttons
                self.key_hist = []
        elif key == pew.K_X:
            # restart the level
            self.__init__()

    def get_current_screen(self):
        return make_image(self.state)

You know already the methods *key_pressed()* and *get_current_screen()* that are required by the main loop. 
In addition, there is the constructor *\__init\__*. 
Its purpose is to set up the "memory" of the level:

 * the variable *self.state* contains the current quantum state
 * the list *self.key_hist* is a memory of the pushed buttons. 
   This is important, because the transformation of the state vector in this level is defined by two rotation axes, one per qubit. 
   Thus, valid inputs are, *xx*, *xy*, *xz*, etc. and the state vector should only be modified after two *key_pressed* events. 

Consequently, the *key_pressed* method appends the values of the pushed buttons to the *key_hist* list until it contains two elements. 
Then, the corresponding transformation is determined, applied to the state vector, and *key_hist* is reset. 

Finally, there is the method *get_current_screen* which calls an auxiliary function *make_image* to convert the current state vector to a frame of the screen. 

Congratulations, now you know all that is necessary to develop your own levels for the PewPew Qube game. 
If you can't wait to start coding, go ahead! 
If you feel you need more details on the *aether* library, read on to get more information on designing quantum games. 

## Writing a level of a quantum game

In this section, we discuss the methods *random_state*, *make_circuit*, *propagate_statevector*, and *make_image* defined in <a href="../src/rotations.py">rotations.py</a> and <a href="../src/propagate_statevector.py">propagate_statevector.py</a>. 
They are used to transform a two-qubit state vector by applying quantum gates to it depending on the user's input. 

The heart of our quantum game is the function *make_circuit*. 
It creates a new quantum circuit *qc* that rotates the two-qubit state around the axes specified in the two-character string *gate*:

In [ ]:
from aether import QuantumCircuit

pi4 = 0.785398
pi2 = 1.570796

def make_circuit(gate):
    qc = QuantumCircuit(2)
    
    if gate[0] == 'x':
        qc.h(0)
    elif gate[0] == 'y':
        qc.rx(pi2,0)
    
    if gate[1] == 'x':
        qc.h(1)
    elif gate[1] == 'y':
        qc.rx(pi2,1)
    
    qc.cx(0,1)
    qc.h(1)
    qc.rx(pi2,1)
    qc.h(1)
    qc.cx(0,1)
    
    if gate[0] == 'x':
        qc.h(0)
    elif gate[0] == 'y':
        qc.rx(-pi2,0)
    
    if gate[1] == 'x':
        qc.h(1)
    elif gate[1] == 'y':
        qc.rx(-pi2,1)
    
    return qc

Once a quantum circuit *qc* is defined, the function *propagate_statevector* can be used to applythe circuit to a state vector *vec*: 

In [ ]:
from aether import QuantumCircuit, simulate

def propagate_statevector(vec,qc):
    qc_i = QuantumCircuit(2,0)
    qc_i.initialize(vec)

    return simulate(qc_i + qc, get='statevector')

Combining these two functions allows us to generate a random statevector by applying, say, 5 randomly chosen quantum circuits to the initial statevector $\vert 00 \rangle$:

In [ ]:
def random_state():
    state = [[1.0,0.0],[0.0,0.0],[0.0,0.0],[0.0,0.0]]
    
    for i in range(5):
        gate = ['xx','xy','xz','yx','yz','yy','zx','zy','zz'][randint(0,8)]
        qc = make_circuit(gate)
        state = propagate_statevector(state, qc)
    return state

The only missing ingredient of our quantum level is a prescription to visualize the state vector. 
This is accomplished by the function *make_image*: 

In [ ]:
def make_image(state):
    blocks = []
    for num in state:
        blocks.append(make_block(num))
    
    image = pew.Pix()
    
    for i in range(2):
        for j in range(4):
            tmp = blocks[2*i][j] + blocks[2*i+1][j]
            for x in range(8):
                image.pixel(x,4*i+j,tmp[x])
    
    return image 

This function iterates the four complex probability amplitudes of to the states $\vert 00 \rangle$, $\vert 01 \rangle$, $\vert 10 \rangle$, and $\vert 11 \rangle$ and converts them into 4x4 blocks. 
In turn, these four blocks are then combined to a single 8x8 image to be displayed. 
The *make_block* function first determines the magnitude and phase of each probability amplitude. 
The amplitude determines the shape of the output pattern, whereas the phase determines its orientation:

In [ ]:
def make_block(c_num):
    amp = sqrt(c_num[0]*c_num[0] + c_num[1]*c_num[1])
    phi = atan2(c_num[1], c_num[0])
    
    if amp < 0.01:
        phi = 0
    
    scenario = 0
    phases = [0.0, pi4, -pi4, 2.0*pi4, -2.0*pi4, 3.0*pi4, -3.0*pi4, 4.0*pi4, -4.0*pi4]
    scenarios = [1,  -1,   -2,   4,        2,      -4,       -3,       3,        3      ]
    for i in range(9):
        if (phi - phases[i])*(phi - phases[i]) < 0.001:
            scenario = scenarios[i]
            continue
    
    if amp < 0.25:
        block = [[0,0,0,0],[0,2,2,0],[0,2,2,0],[0,0,0,0]]
    elif amp < 0.6:
        if scenario > 0:
            block = [[0,0,2,0],[0,0,0,2],[0,0,0,2],[0,0,2,0]]
        else:
            block = [[0,0,0,0],[0,2,2,0],[0,0,2,0],[0,0,0,0]]
    elif amp < 0.9:
        if scenario > 0:
            block = [[0,0,0,2],[0,0,2,0],[0,0,2,0],[0,0,0,2]]
        else:
            block = [[0,0,2,0],[0,0,2,2],[0,0,0,0],[0,0,0,0]]
    else:
        if scenario > 0:
            block = [[0,0,0,2],[0,0,0,2],[0,0,0,2],[0,0,0,2]]
        else:
            block = [[0,0,2,2],[0,0,0,2],[0,0,0,0],[0,0,0,0]]
    
    if scenario != 1 and scenario != -1:
        for r in range(abs(scenario) - 1):
            block = rot90(block)
    
    return block

Rotations of the pattern by $90$ degrees are done with the following method:

In [ ]:
def rot90(block):
    res = []
    for i in range(len(block)):
        transposed = []
        for col in block:
            transposed.append(col[i])
        transposed.reverse()
        res.append(transposed)
    return res

The levels 1-3 of PewPew Qube are defined by the instruction set *InstructionSet* contained in <a href="../src/instruction_sets.py">instruction_sets.py</a>. 
They implement a different visualization of the state vector based on permutations (defined in <a href="../src/permute_screen.py">permute_screen.py</a>) of an image (defined in <a href="../src/displays.py">displays.py</a>). 